In [1]:
using Revise
using LinearAlgebra
using AdaptiveTransportMap
using Statistics
using Distributions
using Test
using ForwardDiff
using SpecialFunctions

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1278
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116
┌ Warning: Type annotations on keyword arguments not currently supported in recipes. Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/92zOw/src/RecipesBase.jl:116


In [16]:
Nx = 3
Ny = 3
Ne = 20

X0 = Matrix([6.848990102045981  10.771412378771835  17.251353666504375
   7.335533068257697  11.431774819000315  18.008631032494343
   6.893530532930235  10.814513877209178  17.402637463059015
   6.982483262982974  10.955566308126610  17.585507498250752
   7.772972373718459  11.897364353970229  18.806751178470250
   6.729861122793936  10.627146166859600  17.136735985971175
   7.165141335467367  11.237044564345492  17.700384237168816
   7.677696221932205  11.841705192530075  18.555997547057114
   6.954564707114959  10.841937556898699  17.713388581476089
   7.281189323846219  11.429366045803370  17.720746732284457
   6.165359268851407   9.814580026125448  16.494654643919819
   6.899980216434494  10.897939712156095  17.384424698380467
   6.976262366305364  11.026281519121600  17.402194038456781
   7.176370766253880  11.168923654587891  17.805515688591250
   7.606077449251913  11.759630810671281  18.442698417688018
   7.848066004442097  12.077837441717850  18.729219513830220
   7.397949033652097  11.460504706427054  18.273620993443956
   7.303891357733692  11.374927869706132  17.958508019194390
   7.505805036950448  11.581428128513624  18.382815639125237
   7.785289363611799  11.939510743120504  18.759353378485837]')

X = vcat(zeros(Ny, Ne), X0)

6×20 Array{Float64,2}:
  0.0       0.0       0.0       0.0      …   0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  0.0       0.0       0.0       0.0          0.0       0.0       0.0
  6.84899   7.33553   6.89353   6.98248      7.30389   7.50581   7.78529
 10.7714   11.4318   10.8145   10.9556      11.3749   11.5814   11.9395
 17.2514   18.0086   17.4026   17.5855   …  17.9585   18.3828   18.7594

In [17]:
Δtdyn = 0.05
Δtobs = 0.2

0.2

In [18]:
h(x, t) = x
F = StateSpace(lorenz63!, h)

StateSpace(AdaptiveTransportMap.lorenz63!, h)

In [19]:
### Process and observation noise
σx = 1e-6
σy = 1e-6

ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
ϵy = AdditiveInflation(Ny, zeros(Ny), σy)

AdditiveInflation(3, [0.0, 0.0, 0.0], [1.0e-12 0.0 0.0; 0.0 1.0e-12 0.0; 0.0 0.0 1.0e-12], [1.0e-6 0.0 0.0; 0.0 1.0e-6 0.0; 0.0 0.0 1.0e-6])

In [20]:
p = 1
# order = [[-1], [1; 1], [-1; 1; 0], [-1; 1; 1; 0]]
order = [[-1], [p; p], [-1; p; 0], [-1; p; p; 0]]

# parameters of the radial map
γ = 2.0
λ = 0.0
δ = 1e-8
κ = 4.0
β = sqrt(1.1)

dist = Float64.(metric_lorenz(3))
idx = vcat(collect(1:Ny)',collect(1:Ny)')

smf = SparseRadialSMF(x->x, F.h, β, ϵy, order, γ, λ, δ, κ, 
                      Ny, Nx, Ne, 
                      Δtdyn, Δtobs, 
                      dist, idx; islocalized = true)

SparseRadialSMF(var"#3#4"(), h, 1.0488088481701516, AdditiveInflation(3, [0.0, 0.0, 0.0], [1.0e-12 0.0 0.0; 0.0 1.0e-12 0.0; 0.0 0.0 1.0e-12], [1.0e-6 0.0 0.0; 0.0 1.0e-6 0.0; 0.0 0.0 1.0e-6]), Sparse Radial Map of dimension Nx = 4 and order p = [[-1], [1, 1], [-1, 1, 0], [-1, 1, 1, 0]]
        with parameters (γ, λ, δ, κ) = (2.0, 0.0, 1.0e-8, 4.0)
, 3, 3, 0.05, 0.2, [0.0 1.0 1.0; 1.0 0.0 1.0; 1.0 1.0 0.0], [1 2 3; 1 2 3], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], false, true)

In [21]:
smf.S.κ

4.0

In [22]:
ystar = [-0.508920322604891;-1.429075047747251;0.299044085894253]

3-element Array{Float64,1}:
 -0.508920322604891
 -1.429075047747251
  0.299044085894253

In [23]:
mean(X; dims = 2);

In [24]:
i = 1

1

In [25]:
smf.dist

3×3 Array{Float64,2}:
 0.0  1.0  1.0
 1.0  0.0  1.0
 1.0  1.0  0.0

In [26]:
idx = smf.idx

2×3 Array{Int64,2}:
 1  2  3
 1  2  3

In [28]:
X1 = smf(deepcopy(X), ystar[i], 0.0, idx[:,i])

perm = [1, 2, 3]
norm(cache[1, :]) = 32.32602637065226
norm(cache[2, :]) = 32.326026600316645
norm(cache[3, :]) = 50.36278002934032
norm(cache[4, :]) = 79.9930454225353
cache[1, :] = [6.831107826809802, 7.341398089667187, 6.877823095160802, 6.971117042305891, 7.800189199747472, 6.706164733947984, 7.162692077417925, 7.70026317768785, 6.941835687011849, 7.284401046268628, 6.114110812608025, 6.884587889557894, 6.96459407618227, 7.174467392043636, 7.6251477393633085, 7.8789482223297895, 7.406861748606554, 7.308212727902754, 7.519982286707538, 7.813107128720145]
cache[2, :] = [6.831108465893829, 7.341399033871628, 6.877822863906546, 6.97111727425475, 7.800189247976281, 6.706164938380805, 7.162690676866085, 7.700262776963255, 6.941836045832242, 7.284402833890172, 6.114110399157427, 6.884587349033709, 6.96459274277572, 7.174468203234895, 7.625148374481078, 7.8789481123204546, 7.406861450644357, 7.3082129279028125, 7.519981781231936, 7.813107415959239]
cache[3, :] = [10.748176564682273, 11.440

3×20 view(::Array{Float64,2}, [1, 2, 3], :) with eltype Float64:
 -0.46577  -0.46574  -0.465785  -0.465795  …  -0.465744  -0.465755  -0.465795
  1.29896   1.31689   1.28046    1.29986       1.29951    1.25943    1.292
  7.02567   7.10135   7.12354    7.18013       7.0972     7.19978    7.1184

In [178]:
xp1 =   [-0.465576124088329   1.299206384812404   7.025942132323366
  -0.465545300633494   1.317135750832397   7.101623221820033
  -0.465590672975850   1.280704611080655   7.123815440893266
  -0.465600823419663   1.300113550327429   7.180408739163755
  -0.465605399657761   1.264216987280329   7.176603849561077
  -0.465520288869811   1.321427238836452   7.059558616310834
  -0.465575695653570   1.338915074997654   7.038758840490709
  -0.465606259844208   1.318260440409364   7.105760906742069
  -0.465599997766731   1.224447235375511   7.368684897008732
  -0.465552389052391   1.382721403660105   6.898639260158044
  -0.465496536770843   1.300834899492525   7.135884663932835
  -0.465593075410652   1.355244526007959   7.073974812031393
  -0.465601653632802   1.379268427552456   6.986816632191978
  -0.465573902105461   1.256223646404759   7.134714607090130
  -0.465591024342757   1.319500624685293   7.111287253918062
  -0.465559895397746   1.358809510136506   7.008600563603973
  -0.465546658340827   1.268942826817906   7.265639042860513
  -0.465550042964423   1.299758597545847   7.097474332266549
  -0.465562062948266   1.259677997170126   7.200052137348411
  -0.465601634651648   1.292250252441333   7.118680155090951]

20×3 Array{Float64,2}:
 -0.465576  1.29921  7.02594
 -0.465545  1.31714  7.10162
 -0.465591  1.2807   7.12382
 -0.465601  1.30011  7.18041
 -0.465605  1.26422  7.1766
 -0.46552   1.32143  7.05956
 -0.465576  1.33892  7.03876
 -0.465606  1.31826  7.10576
 -0.4656    1.22445  7.36868
 -0.465552  1.38272  6.89864
 -0.465497  1.30083  7.13588
 -0.465593  1.35524  7.07397
 -0.465602  1.37927  6.98682
 -0.465574  1.25622  7.13471
 -0.465591  1.3195   7.11129
 -0.46556   1.35881  7.0086
 -0.465547  1.26894  7.26564
 -0.46555   1.29976  7.09747
 -0.465562  1.25968  7.20005
 -0.465602  1.29225  7.11868

In [ ]:
6.83110884947197, 7.341398612606025, 6.877821606400789, 6.97111718713474, 7.800188427327412, 6.706164965915386, 7.162691765378075, 7.700262635733485, 6.941836035397618, 7.284399835461074, 6.1141089743101045, 6.884585805177348, 6.964593643958426, 7.17446934644567, 7.625147346652602, 7.8789482626118605, 7.406861110727726, 7.308211351908329, 7.519981073763811, 7.813106809359723

In [163]:
Xopt = [6.83111057214752 7.341399483035104 6.877822720168037 6.971118924033331 7.80018915888386 6.706163613626499  7.162690594670549  7.700261382011425  6.941835922612477  7.284403548637827  6.114109300787112  6.8845869963952495  6.964592565400086  7.174468426637868  7.625149047410672  7.878949787131567  7.406859873033216  7.308213228925083  7.519981945292273  7.813106261096211;
        6.831108465893829  7.341399033871628  6.877822863906546  6.97111727425475  7.800189247976281 6.706164938380805  7.162690676866085  7.700262776963255  6.941836045832242  7.284402833890172  6.114110399157427  6.884587349033709  6.96459274277572  7.174468203234895  7.625148374481078  7.8789481123204546  7.406861450644357  7.3082129279028125  7.519981781231936  7.813107415959239;
    10.748176564682273  11.440770534993137  10.793381797612751  10.941318835214862  11.929084958885008  10.59686888513673  11.236535720904731  11.870709137884843  10.822143995520504  11.438244192350945  9.74464232701128  10.88087955147106  11.015485574206627  11.165089908005559  11.784628799983244  12.118366730171301  11.470902695133232  11.381148951581917  11.59772825016864  11.97328846474424;
    17.220877261555913  18.015116463525178  17.37954504596719  17.571340756941474  18.85219193452756  17.100665224055952  17.69182449716767  18.58919930719067  17.70546356854179  17.713180862215776  16.427244631267527  17.36044333722269  17.379079978320874  17.80208729364049  18.470370177818385  18.770876038639738  18.293040279245393  17.962547003679237  18.407564589808725  18.80248070252011]

4×20 Array{Float64,2}:
  6.83111   7.3414   6.87782   6.97112  …   7.30821   7.51998   7.81311
  6.83111   7.3414   6.87782   6.97112      7.30821   7.51998   7.81311
 10.7482   11.4408  10.7934   10.9413      11.3811   11.5977   11.9733
 17.2209   18.0151  17.3795   17.5713      17.9625   18.4076   18.8025

In [164]:
Xopt = Xopt[1:2,:];

In [165]:
C.ξ

LoadError: [91mtype SparseRadialMap has no field ξ[39m

In [151]:
optimize(S, Xopt)

μψ = [-0.17946312318237084; 0.4973917394053714; 0.17454058209306936]
σψ = [0.18147399463599764; 0.22060093098446143; 0.13572449746649612]
(i, xopt) = (2, [-36976.57962814703, -656.5341280277023, 255902.51188927222, 37229.538860661596, 22473.93334257556, 38568.05490267168])


In [153]:
S = SparseRadialMap(2, [[-1],[p; p]]; γ = 2.0, λ = 0.0, δ = 1e-8, κ = 4.0)


Sparse Radial Map of dimension Nx = 2 and order p = [[-1], [1, 1]]
        with parameters (γ, λ, δ, κ) = (2.0, 0.0, 1.0e-8, 4.0)


In [147]:
center_std(S, Xopt)

In [148]:
S[2].ξ

2-element Array{Array{Float64,1},1}:
 [7.229435987637848]
 [6.913211697432976, 7.229435518562534, 7.572565077856507]

In [149]:
S[2].σ

2-element Array{Array{Float64,1},1}:
 [0.6593540368476098]
 [0.6324476422591161, 0.6593533804235312, 0.6862591185879463]

In [152]:
S[2].a

2-element Array{Array{Float64,1},1}:
 [-36976.57962814703, -656.5341280277023]
 [255902.51188927222, 37229.538860661596, 22473.93334257556, 38568.05490267168]

In [107]:
ψ_off, ψ_mono, dψ_mono = weights(C, Xopt)

LoadError: [91mMethodError: no method matching weights(::RadialMap, ::Array{Float64,2})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  weights(::RadialMap, ::AbstractArray{Float64,2}, [91m::Array{Float64,2}[39m, [91m::Array{Float64,2}[39m, [91m::Array{Float64,2}[39m) at /media/mat/HDD/AdaptiveTransportMap/src/radialmap/weights.jl:223[39m
[91m[0m  weights(::RadialMap, ::Any, [91m::Weights[39m) at /media/mat/HDD/AdaptiveTransportMap/src/radialmap/weights.jl:287[39m
[91m[0m  weights(::RadialMap, [91m::Array{Float64,1}[39m, [91m::Array{Float64,1}[39m, [91m::Array{Float64,1}[39m, [91m::Array{Float64,1}[39m) at /media/mat/HDD/AdaptiveTransportMap/src/radialmap/weights.jl:181[39m
[91m[0m  ...[39m

In [109]:
fieldnames(typeof(W))

(:Nx, :p, :Ne, :woff, :wdiag, :w∂k)

In [110]:
W.woff

2×20 Array{Float64,2}:
 6.83111  7.3414   6.87782   6.97112   …  7.30821   7.51998   7.81311
 0.50413  0.59639  0.524856  0.560354     0.600747  0.549069  0.408915

In [84]:
ψ_mono

3×20 Array{Float64,2}:
 -5.92599e-13  -1.38545e-14  -4.24661e-13  …  -3.71139e-15  -4.57109e-16
  1.0           1.0           1.0              1.0           1.0
  6.83111       7.3414        6.87782          7.51998       7.81311

In [85]:
dψ_mono

3×20 Array{Float64,2}:
 4.21307e-12  1.05693e-13  3.03874e-12  …  2.7367e-14   2.77556e-15
 2.93726e-11  7.89791e-13  2.13245e-11     2.09511e-13  2.21428e-14
 1.0          1.0          1.0             1.0          1.0

In [88]:
ψ_off[2,:]

20-element Array{Float64,1}:
 2.937214447187426e-11
 7.897880963026246e-13
 2.1324569331393987e-11
 1.1176720140505991e-11
 2.4492388713305853e-14
 6.842699266307578e-11
 2.8864122797725495e-12
 5.313443364943918e-14
 1.370196967622856e-11
 1.1981935565200967e-12
 3.0439652765490383e-9
 2.035473427334963e-11
 1.169671205898906e-11
 2.652715892593548e-12
 9.448006539935866e-14
 1.3209192485513104e-14
 4.873834925972258e-13
 1.0071174928083124e-12
 2.0951045296639121e-13
 2.2143038056311838e-14

In [92]:
Xi = Matrix(  [-0.8544   -0.7938
                0.3573    0.5410
                2.7485   -0.5591
               -1.5130    1.9766
                0.4340    0.5447
               -0.2298   -0.1379
               -0.8271    0.6199
               -0.8320   -0.0056
                0.4979    1.1072
                2.3156   -0.1856]')

2×10 Array{Float64,2}:
 -0.8544  0.3573   2.7485  -1.513   …  -0.8271  -0.832   0.4979   2.3156
 -0.7938  0.541   -0.5591   1.9766      0.6199  -0.0056  1.1072  -0.1856

In [93]:
C = SparseRadialMapComponent(2, [1; 1])
λ = 0.0
δ = 1e-8

1.0e-8

In [95]:
optimize(C, Xi, λ, δ)

μψ = [-0.36363804824765433; 0.5857964859079126; 0.6743780482476543]
σψ = [0.2818136776050789; 0.23692272590468227; 0.5349125993854017]


6-element Array{Float64,1}:
 0.3352968883456885
 0.03962302418588382
 0.07698733854242444
 2.2247695651044874
 4.220785474173194e-8
 0.966621043681591

In [98]:
C.ξ


2-element Array{Array{Float64,1},1}:
 [0.0]
 [0.0, 0.0, 0.0]